In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
# train mô hình ML
from sklearn.model_selection import train_test_split
from sklearn import linear_model
# lưu mô hình
import pickle

In [20]:
conn = sqlite3.connect('database.sqlite')
Match = pd.read_sql("""SELECT * FROM Match""", conn)
query = """SELECT Match.id, 
                    Country.name AS country_name, 
                    League.name AS league_name,                     
                    HT.team_long_name AS  home_team,
                    AT.team_long_name AS away_team                                                           
            FROM Match
            JOIN Country on Country.id = Match.country_id
            JOIN League on League.id = Match.league_id
            LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
            LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
            """

detailed_matches = pd.read_sql(query, conn)

In [21]:
pd.set_option('display.max_columns', None) # hiển thị tất cả các cột trong dataframe

In [22]:
detailed_matches = pd.merge(detailed_matches, Match, on='id')

In [23]:
def getResultWinLoss_2(match_info):
    if match_info['home_team_goal'] > match_info['away_team_goal']:
        home_team_result = 3
        away_team_result = 0
    elif match_info['home_team_goal'] < match_info['away_team_goal']:
        home_team_result = 0
        away_team_result = 3
    else: 
        home_team_result = 1
        away_team_result = 1
    return [home_team_result, away_team_result]

detailed_matches[['home_team_result', 'away_team_result']] = detailed_matches[['home_team_goal', 'away_team_goal']].apply(getResultWinLoss_2, axis = 1, result_type='expand')
detailed_matches['dif_goal'] = detailed_matches['home_team_goal'] - detailed_matches['away_team_goal']
detailed_matches.dropna(inplace=True)

In [25]:
X=detailed_matches[[
'B365H',
'B365D',
'B365A'
]]
y=detailed_matches['dif_goal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model=linear_model.LinearRegression()
model.fit(X_train,y_train)

pickle.dump(model, open('modelSav/modelHomeWinWithBet365.sav','wb'))

In [33]:
X=detailed_matches[[
'BSH',
'BSD',
'BSA'
]]
y=detailed_matches['dif_goal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model=linear_model.LinearRegression()
model.fit(X_train,y_train)

pickle.dump(model, open('modelSav/modelHomeWinWithBlueSquare.sav','wb'))